In [1]:
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.multinomial import Multinomial
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import sys
sys.path.append('/home/hao/Research/probtorch/')
import probtorch
from probtorch.util import log_sum_exp
from util_hmm_pytorch import *
from util_data import *
import numpy as np

# output_notebook()

In [26]:
K = 4
N = 500
D = 2
# Y = sampling()
# plot_samples(Y)
# Y = torch.from_numpy(Y)
iterations = 100
T = 500
dt = 1
Boundary = 6 ## unit box is amplified by this value, centered at origin
signal_noise_ratio = 0.2 # the noise is multiplied by this param

init_state = np.random.random(4) * 8
# init_state = np.array([0,0,0.2,-0.1])
Y, A_true, STATE = generate_data(T, dt, init_state, Boundary, signal_noise_ratio)
Y = torch.from_numpy(Y).float()
N, D = Y.shape

In [32]:
## prior
alpha_init_0 = torch.ones(K)
# L = torch.ones((K, K))  / (2 *(K-1))
# alpha_trans_0 = torch.cat((torch.cat((torch.eye(4)*0.5, torch.ones((4, K-4)) * (0.5 / (K-4))), 1),\
# torch.ones((K-4, K)) * (1.0 / K)), 0).double()
alpha_trans_0 = torch.ones((K, K))
m_0 = Y.mean(0)
beta_0 = 1.0
nu_0 = 4.0
W_0 =  (nu_0-D-1) * torch.mm((Y - m_0).transpose(0,1), (Y - m_0)) / (N)
cov = torch.from_numpy(np.cov(Y.transpose(0,1))).float()
## plot transition prior
# a = alpha_trans_0.data.numpy()

In [33]:
alpha_init_hat = alpha_init_0
alpha_trans_hat = alpha_trans_0
m_ks = MultivariateNormal(m_0, cov).sample((K,))
beta_ks = torch.ones(K) * beta_0
nu_ks = torch.ones(K) * nu_0
W_ks = W_0.repeat(K, 1, 1)
# cov_ks = np.tile(cov,(K,1,1)) 

In [34]:
epoch_elbos = []
for i in range(iterations):
    ## VBE step
    log_gammas, log_eta = vbE_step(alpha_init_hat, alpha_trans_hat, nu_ks, W_ks, m_ks, beta_ks, Y, N, D, K)
    N_ks, Y_ks, S_ks = stats(log_gammas, Y, D, K)
    ## VBM step
    alpha_init_hat, alpha_trans_hat, nu_ks, W_ks, m_ks, beta_ks, cov_ks = vbM_step(log_eta, alpha_init_0, alpha_trans_0, nu_0, W_0, m_0, beta_0, N_ks, Y_ks, S_ks, N, D, K)
#     ## elbokl_niw(nu_0, W_0, nu_ks, W_ks, D, K)
#     epoch_elbo = elbo(log_gammas, log_eta, alpha_init_0, alpha_trans_0, nu_0, W_0, m_0, beta_0, N_ks, Y_ks, S_ks, alpha_init_hat, alpha_trans_hat, nu_ks, W_ks, m_ks, beta_ks, Y, N, D, K)
#     epoch_elbos.append(epoch_elbo)
    print('Iteration:%d' % (i+1))
# # final_covs = cov_ks ## mean of inverse wishart
# # final_mus = m_ks ## mean of gaussian

Iteration:1
Iteration:2
Iteration:3
Iteration:4
Iteration:5
Iteration:6
Iteration:7
Iteration:8
Iteration:9
Iteration:10
Iteration:11
Iteration:12
Iteration:13
Iteration:14
Iteration:15
Iteration:16
Iteration:17
Iteration:18
Iteration:19
Iteration:20
Iteration:21
Iteration:22
Iteration:23
Iteration:24
Iteration:25
Iteration:26
Iteration:27
Iteration:28
Iteration:29
Iteration:30
Iteration:31
Iteration:32
Iteration:33
Iteration:34
Iteration:35
Iteration:36
Iteration:37
Iteration:38
Iteration:39
Iteration:40
Iteration:41
Iteration:42
Iteration:43
Iteration:44
Iteration:45
Iteration:46
Iteration:47
Iteration:48
Iteration:49
Iteration:50
Iteration:51
Iteration:52
Iteration:53
Iteration:54
Iteration:55
Iteration:56
Iteration:57
Iteration:58
Iteration:59
Iteration:60
Iteration:61
Iteration:62
Iteration:63
Iteration:64
Iteration:65
Iteration:66
Iteration:67
Iteration:68
Iteration:69
Iteration:70
Iteration:71
Iteration:72
Iteration:73
Iteration:74
Iteration:75
Iteration:76
Iteration:77
Iteratio

In [22]:
a = output_transition(alpha_trans_hat)
# fig=plt.figure(figsize=(8, 8))
# fig.add_subplot(1, 2, 1)
# plt.imshow(a, cmap='viridis', vmin=0, vmax=1.0)
# # fig.add_subplot(1, 2, 2)
# plt.imshow(A_true, cmap='viridis', vmin=0, vmax=1)

In [35]:
## sample from initial distribution
final_init_prob = alpha_init_hat / alpha_init_hat.sum()
init_obj = Multinomial(1, final_init_prob)
init_hidden = torch.nonzero(init_obj.sample()).item()
## transition matrix
A = (alpha_trans_hat.transpose(0,1) / alpha_trans_hat.sum(1)).transpose(0,1)
plt.imshow(A)
plt.colorbar()

RuntimeError: invalid argument 2: invalid multinomial distribution (encountering probability entry < 0) at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/TH/generic/THTensorRandom.cpp:298

In [ ]:
plt.scatter(m_ks[:,0], m_ks[:,1])

In [19]:
A

tensor([[0.8780, 0.0512, 0.0039, 0.0669],
        [0.0960, 0.6640, 0.0080, 0.2320],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.1278, 0.2180, 0.0075, 0.6466]])